In [5]:
import pandas as pd

# Load merged, cleaned data
df = pd.read_csv("orders_full.csv", parse_dates=['order_purchase_timestamp', 'order_delivered_customer_date'])

In [6]:
df['order_delivered_customer_date'] = pd.to_datetime(df['order_delivered_customer_date'], errors='coerce')
df['order_estimated_delivery_date'] = pd.to_datetime(df['order_estimated_delivery_date'], errors='coerce')


# Handle delivery delay (days)
df['delivery_delay'] = (df['order_delivered_customer_date'] - df['order_estimated_delivery_date']).dt.days
df['delivery_delay'] = df['delivery_delay'].fillna(0).astype(int)

In [7]:
# Is the order late?
df['is_late'] = df['delivery_delay'].apply(lambda x: 1 if x > 0 else 0)

In [8]:
# Delivery duration (from purchase to delivery)
df['delivery_duration'] = (df['order_delivered_customer_date'] - df['order_purchase_timestamp']).dt.days

In [9]:
# Extract month (for seasonality, BI filtering)
df['order_month'] = df['order_purchase_timestamp'].dt.to_period('M').astype(str)

In [10]:
# Number of items per order
item_counts = df.groupby('order_id')['order_item_id'].count().rename('num_items')
df = df.merge(item_counts, on='order_id', how='left')

In [11]:
# Total order value
df['order_value'] = df['price'] + df['freight_value']

In [12]:
# Average order value per customer (enrichment)
aov_per_customer = df.groupby('customer_id')['order_value'].mean().rename('aov_customer')
df = df.merge(aov_per_customer, on='customer_id', how='left')

In [13]:
final_df = df[[
    'order_id',
    'customer_id',
    'customer_state',
    'order_month',
    'product_category_name',
    'order_value',
    'freight_value',
    'delivery_duration',
    'delivery_delay',
    'is_late',
    'review_score',
    'num_items',
    'aov_customer'
]]

In [14]:
# Save to enriched CSV
final_df.to_csv("olist_model_ready.csv", index=False)

# Quick preview
final_df.head()

,order_id,customer_id,customer_state,order_month,product_category_name,order_value,freight_value,delivery_duration,delivery_delay,is_late,review_score,num_items,aov_customer
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,SP,2017-10,utilidades_domesticas,38.71,8.72,8.0,-8,0,4.0,1,38.71
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,BA,2018-07,perfumaria,141.46,22.76,13.0,-6,0,4.0,1,141.46
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,GO,2018-08,automotivo,179.12,19.22,9.0,-18,0,5.0,1,179.12
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,RN,2017-11,pet_shop,72.20,27.20,13.0,-13,0,5.0,1,72.20
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,SP,2018-02,papelaria,28.62,8.72,2.0,-10,0,5.0,1,28.62
